# This notebook converts a trained Pytorch model to a TensorFlow version of the model.
More details can be found at https://github.com/onnx/tutorials/blob/master/tutorials/PytorchTensorflowMnist.ipynb

### Throughout the notebook, there are several spots where a file path or file name need to be specified for the processing of each model. Those locatations are inticated by '***'. Remove the *'s and instert the name without changing the file extension (.pth, .onnx, .pb). 

In [ ]:
import tensorflow as tf
import torch
import onnx
from onnx_tf.backend import prepare
import numpy as np
import os
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

### Saves the weights for the model/model custimization
Specify the path to the model in torch.load().

In [ ]:
loc = torch.load(path/'***.pth')
body = create_body(models.resnet101, True, None)
data_classes = 2
nf = callbacks.hooks.num_features_model(body) * 2
head = create_head(nf, data_classes, None, ps=0.5, bn_final=False)
model = nn.Sequential(body, head)

### Load Model

In [ ]:
trained_model = model.load_state_dict(loc['model'])

### Export the trained model to ONNX
In order to export the model, Pytorch exporter needs to run the model once and save this resulting trained model to a ONNX file. Therefore, we need to provide the input for the model we hope to convert. 

In [ ]:
dummy_input = Variable(torch.randn(1, 3, 28, 28)) #(# of images, options for pixels should be one for b/w images and 3 for rgb images, x dimension, y diemsion) one rgb 28 x 28 picture will be the input to the model
torch.onnx.export(model, dummy_input, path/"***.onnx", input_names=['input'], output_names=['output'])

### Load the ONNX file and import the model to Tensorflow

In [ ]:
model = onnx.load('***.onnx')

tf_rep = prepare(model)

### Print statements to verify the model converted correctly 

In [ ]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

### Save the Tensorflow model to a file. 
You man rename the file, just keep the .pb file extenstion

In [ ]:
tf_rep.export_graph('***.pb')